In [1]:
import math
import numpy as np
import h5py
# import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
import json
import random
from collections import OrderedDict
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# %matplotlib inline
# np.random.seed(0)

In [2]:
import cv2

In [3]:
EPOCHS = 10
BATCH_SIZE = 32
IMG_SIZE = 224
data_file = "dataset.txt"

In [4]:
weight_file ="vgg16_weights.npz"
weights = np.load(weight_file)

In [5]:
keys = sorted(weights.keys())
# for index, key in enumerate(keys):
#     print(index, key, np.shape(weights[key]))

0 conv1_1_W (3, 3, 3, 64)
1 conv1_1_b (64,)
2 conv1_2_W (3, 3, 64, 64)
3 conv1_2_b (64,)
4 conv2_1_W (3, 3, 64, 128)
5 conv2_1_b (128,)
6 conv2_2_W (3, 3, 128, 128)
7 conv2_2_b (128,)
8 conv3_1_W (3, 3, 128, 256)
9 conv3_1_b (256,)
10 conv3_2_W (3, 3, 256, 256)
11 conv3_2_b (256,)
12 conv3_3_W (3, 3, 256, 256)
13 conv3_3_b (256,)
14 conv4_1_W (3, 3, 256, 512)
15 conv4_1_b (512,)
16 conv4_2_W (3, 3, 512, 512)
17 conv4_2_b (512,)
18 conv4_3_W (3, 3, 512, 512)
19 conv4_3_b (512,)
20 conv5_1_W (3, 3, 512, 512)
21 conv5_1_b (512,)
22 conv5_2_W (3, 3, 512, 512)
23 conv5_2_b (512,)
24 conv5_3_W (3, 3, 512, 512)
25 conv5_3_b (512,)
26 fc6_W (25088, 4096)
27 fc6_b (4096,)
28 fc7_W (4096, 4096)
29 fc7_b (4096,)
30 fc8_W (4096, 1000)
31 fc8_b (1000,)


In [6]:
def conv_layer(input_data,filter_size,stride_size,pad="SAME",name="conv",weight_name="W1",bias_name="b1"):
    print("creating layer :"+name)
    with tf.name_scope(name):
        W = tf.Variable(weights[name+"_W"],name="weight")
        b = tf.Variable(weights[name+"_b"],name="bias")
        
        conv = tf.nn.conv2d(input_data, W, strides=stride_size, padding=pad)
        out = tf.nn.bias_add(conv, b)
        act = tf.nn.relu(out)
        tf.summary.histogram("weights", W)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [7]:
def vggnet_v1(input_data):
    print("creating model")
    output=[]
    layer_input = input_data
    
    with tf.name_scope('preprocess') as scope:
        mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
        layer_input = layer_input-mean
            
    print("reading vgg16 json file")
    vgg = json.load(open("vgg16.json"),object_pairs_hook=OrderedDict)
       
    for index,layer in enumerate(vgg):
        if "conv" in layer:
            output = conv_layer(layer_input,vgg[layer]["weights"],vgg[layer]["stride"],vgg[layer]["pad"],layer,"weight"+str(index),"bias"+str(index))
        elif "pool" in layer:
            output = tf.nn.max_pool(layer_input, ksize=vgg[layer]["ksize"], strides=vgg[layer]["strides"], padding=vgg[layer]["padding"],name=layer)
        layer_input = output
    
    pool5_flat = tf.contrib.layers.flatten(output)

    print("creating layer : fc6")
    with tf.name_scope('fc6'):
        fc6_W = tf.Variable(weights["fc6_W"],name="fc6_W")
        fc6_b= tf.Variable(weights["fc6_b"],name="fc6_b")

        fc6 = tf.nn.bias_add(tf.matmul(pool5_flat, fc6_W), fc6_b)
        fc6 = tf.nn.relu(fc6)
    
#     fc6 = tf.contrib.layers.fully_connected(P3,4096, weights_initializer=fc6_W, biases_initializer=fc6_b, scope="fc6")
    tf.summary.histogram("fc6/relu", fc6)
    
    dropout1 = tf.layers.dropout(inputs=fc6, rate=0.5)

    print("creating layer : fc7")
    with tf.name_scope('fc7'):
        fc7_W = tf.Variable(weights["fc7_W"],name="fc7_W")
        fc7_b= tf.Variable(weights["fc7_b"],name="fc7_b")

        fc7 = tf.nn.bias_add(tf.matmul(dropout1, fc7_W), fc7_b)
        fc7 = tf.nn.relu(fc7)
    
#     fc7 = tf.contrib.layers.fully_connected(dropout1,4096, weights_initializer=fc7_W, biases_initializer=fc7_b, scope="fc7")
    tf.summary.histogram("fc7/relu", fc7)
    
    dropout2 = tf.layers.dropout(inputs=fc7, rate=0.5)
    
    print("creating layer : fc8")
    fc8 = tf.contrib.layers.fully_connected(dropout2,2, biases_initializer=tf.constant_initializer(1.0), activation_fn=None, scope="fc8")
    tf.summary.histogram("fc8", fc8)
    
    
    print("model completed.")
    ##NEED FULLY CONNECTED LAYER
    return fc8

In [8]:
mean = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))
print(mean)

[[[[ 123.68   116.779  103.939]]]]


In [9]:
def prepare_data(data):
    newData=[]
    labels=[]
    print("preparing data....")
    for sample in data:
        img_path,label = sample.strip().split(" ")
        img = cv2.imread(img_path)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        #img = Image.open(img_path)
        #img = img.resize((IMG_SIZE,IMG_SIZE))
        img = np.array(img)
        #img = img/255
        newData.append(img)
        if "cat" in img_path:
            labels.append(np.array([1,0]))
        elif "dog" in img_path:
            labels.append(np.array([0,1]))
    print("preparing data completed.")
    return np.array(newData),np.array(labels)

In [10]:
f = open(data_file,"r")
data = f.readlines()
random.shuffle(data)
print(len(data))
train_data = data[:20000]
train_X, train_Y = prepare_data(train_data)
val_data = data[20000:]
val_X, val_Y = prepare_data(val_data)

25000
preparing data....
preparing data completed.
preparing data....
preparing data completed.


In [11]:
# train_X = train_X - mean
# val_X = val_X - mean

In [12]:
# train_X.shape

In [13]:
tf.reset_default_graph()

start_learning_rate = 0.01

batch = tf.Variable(0, trainable=False)
train_size = 20000
val_size = 5000

learning_rate = tf.train.exponential_decay(
  0.001,                # Base learning rate.
  batch * BATCH_SIZE,  # Current index into the dataset.
  train_size,          # Decay step.
  0.1,                # Decay rate.
  staircase=True)

with tf.name_scope('inputs'):
    X = tf.placeholder(tf.float32, shape=(None,IMG_SIZE,IMG_SIZE , 3),name="X")
    tf.summary.image('input', X, BATCH_SIZE)
    Y = tf.placeholder(tf.float32,shape=(None,2),name="labels")
    
logits = vggnet_v1(X)

with tf.name_scope("loss"):    
    train_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name="loss")
    tf.summary.scalar("loss", train_loss)

with tf.name_scope("train"):
    #optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(train_loss)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(train_loss,global_step=batch)
print("optimizer set")

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)
print("accuracy set")

merged_summary = tf.summary.merge_all()

# init = 
# print("initializing the variables")

with tf.Session() as sess:
    print("session created.")
    print("initializing global variables")
    sess.run(tf.global_variables_initializer())
    
    print("writing tensorboard")
    writer = tf.summary.FileWriter("/tmp/tensorboard/cat-dog-vgg1")
    writer.add_graph(sess.graph)
    print("writer added to graph")
    
    for epoch in range(EPOCHS):
        
        print("processing epoch "+str(epoch))
        print("total number of batches in 1 epoch: "+ str(len(train_data)/BATCH_SIZE))
        
        for i in range(0,train_size,BATCH_SIZE): 
            #print(i)
            print("Batch:" + str((i/BATCH_SIZE)+1))
            
            #minibatch_X, minibatch_Y = prepare_data(train_data[i:i+BATCH_SIZE])
            #sess.run(optimizer,feed_dict={X:minibatch_X,Y:minibatch_Y})
            minibatch_X = train_X[i : i + BATCH_SIZE]
            minibatch_Y = train_Y[i : i + BATCH_SIZE]
            _ , temp_loss, temp_accuracy = sess.run([optimizer,train_loss,accuracy],feed_dict={X:minibatch_X,Y:minibatch_Y})
            
            if i % (BATCH_SIZE*10) == 0 and i >200:  # Record summaries and test-set accuracy
                summary = sess.run(merged_summary, feed_dict={X:minibatch_X,Y:minibatch_Y})
                writer.add_summary(summary, i + epoch*len(train_data))
            
            #writer.add_summary(summary, i)
            print("temp_loss: "+str(temp_loss))
            
            if i % (BATCH_SIZE*50) == 0 and i>=1000:
                print("validating...")
                val_loss = 0
                val_accuracy = 0
                val_batch_size = 50
                val_batches = val_size/val_batch_size
                for v in range(0,val_size,val_batch_size):
                    #val_x,val_y = prepare_data(val_data[v:v+BATCH_SIZE])
                    val_minibatch_x = val_X[v : v + val_batch_size]
                    val_minibatch_y = val_Y[v : v + val_batch_size]
                    temp_loss, temp_accuracy = sess.run([train_loss,accuracy],feed_dict={X:val_minibatch_x,Y:val_minibatch_y})
                    val_accuracy+=temp_accuracy
                    val_loss+=temp_loss
                print("validation loss:")
                print(val_loss/val_batches)
                print("validation accuracy: ")
                print(val_accuracy/val_batches)
#             print("temp_accuracy: "+str(temp_accuracy))
#     train_accuracy = accuracy.eval({X: train_data, Y: train_labels})
#     val_data, val_labels = prepare_data(val_data)
#     val_accuracy = accuracy.eval({X: val_data, Y: val_labels})
# #     print("Train Accuracy:", train_accuracy)
#     print("Validation Accuracy:", val_accuracy)
    


creating model
reading vgg16 json file
creating layer :conv1_1
creating layer :conv1_2
creating layer :conv2_1
creating layer :conv2_2
creating layer :conv3_1
creating layer :conv3_2
creating layer :conv3_3
creating layer :conv4_1
creating layer :conv4_2
creating layer :conv4_3
creating layer :conv5_1
creating layer :conv5_2
creating layer :conv5_3
creating layer : fc6
creating layer : fc7
creating layer : fc8
model completed.
optimizer set
accuracy set
session created.
initializing global variables
writing tensorboard
writer added to graph
processing epoch 0
total number of batches in 1 epoch: 625.0
Batch:1.0
temp_loss: 1.50267
Batch:2.0
temp_loss: 88.0442
Batch:3.0
temp_loss: 37.1074
Batch:4.0
temp_loss: 20.2841
Batch:5.0
temp_loss: 190.575
Batch:6.0
temp_loss: 59.754
Batch:7.0
temp_loss: 1.11208
Batch:8.0
temp_loss: 40.6696
Batch:9.0
temp_loss: 3.16134
Batch:10.0
temp_loss: 2.82734
Batch:11.0
temp_loss: 3.40851
Batch:12.0
temp_loss: 2.76492
Batch:13.0
temp_loss: 1.08887
Batch:14.0
t

temp_loss: 0.763792
Batch:233.0
temp_loss: 0.611214
Batch:234.0
temp_loss: 0.637005
Batch:235.0
temp_loss: 0.756088
Batch:236.0
temp_loss: 0.696954
Batch:237.0
temp_loss: 0.690773
Batch:238.0
temp_loss: 0.849046
Batch:239.0
temp_loss: 0.607813
Batch:240.0
temp_loss: 0.847741
Batch:241.0
temp_loss: 0.65179
Batch:242.0
temp_loss: 0.724382
Batch:243.0
temp_loss: 0.672589
Batch:244.0
temp_loss: 0.708153
Batch:245.0
temp_loss: 0.705582
Batch:246.0
temp_loss: 0.655649
Batch:247.0
temp_loss: 0.620115
Batch:248.0
temp_loss: 0.681225
Batch:249.0
temp_loss: 0.676537
Batch:250.0
temp_loss: 0.693759
Batch:251.0
temp_loss: 0.731337
validating...
validation loss:
0.658160487413
validation accuracy: 
0.600799984932
Batch:252.0
temp_loss: 0.684832
Batch:253.0
temp_loss: 0.662968
Batch:254.0
temp_loss: 0.667295
Batch:255.0
temp_loss: 0.630754
Batch:256.0
temp_loss: 0.645387
Batch:257.0
temp_loss: 0.720692
Batch:258.0
temp_loss: 0.671479
Batch:259.0
temp_loss: 0.656581
Batch:260.0
temp_loss: 0.641607
Ba

temp_loss: 0.628268
Batch:478.0
temp_loss: 0.52949
Batch:479.0
temp_loss: 0.60937
Batch:480.0
temp_loss: 0.579871
Batch:481.0
temp_loss: 0.585735
Batch:482.0
temp_loss: 0.635195
Batch:483.0
temp_loss: 0.792846
Batch:484.0
temp_loss: 0.533478
Batch:485.0
temp_loss: 0.86386
Batch:486.0
temp_loss: 0.575443
Batch:487.0
temp_loss: 0.639459
Batch:488.0
temp_loss: 0.632576
Batch:489.0
temp_loss: 0.639743
Batch:490.0
temp_loss: 0.701971
Batch:491.0
temp_loss: 0.734323
Batch:492.0
temp_loss: 0.680941
Batch:493.0
temp_loss: 0.594144
Batch:494.0
temp_loss: 0.628321
Batch:495.0
temp_loss: 0.614527
Batch:496.0
temp_loss: 0.700437
Batch:497.0
temp_loss: 0.625794
Batch:498.0
temp_loss: 0.634725
Batch:499.0
temp_loss: 0.606404
Batch:500.0
temp_loss: 0.702762
Batch:501.0
temp_loss: 0.695777
validating...
validation loss:
0.631963432729
validation accuracy: 
0.639599981904
Batch:502.0
temp_loss: 0.646394
Batch:503.0
temp_loss: 0.654474
Batch:504.0
temp_loss: 0.588511
Batch:505.0
temp_loss: 0.525419
Batc

temp_loss: 0.730819
validating...
validation loss:
0.562566081285
validation accuracy: 
0.714799987078
Batch:102.0
temp_loss: 0.632714
Batch:103.0
temp_loss: 0.510193
Batch:104.0
temp_loss: 0.434485
Batch:105.0
temp_loss: 0.843036
Batch:106.0
temp_loss: 0.62962
Batch:107.0
temp_loss: 0.663804
Batch:108.0
temp_loss: 0.494912
Batch:109.0
temp_loss: 0.72871
Batch:110.0
temp_loss: 0.749789
Batch:111.0
temp_loss: 0.697772
Batch:112.0
temp_loss: 0.57421
Batch:113.0
temp_loss: 0.684951
Batch:114.0
temp_loss: 0.513881
Batch:115.0
temp_loss: 0.763324
Batch:116.0
temp_loss: 0.707273
Batch:117.0
temp_loss: 0.759448
Batch:118.0
temp_loss: 0.63777
Batch:119.0
temp_loss: 0.488398
Batch:120.0
temp_loss: 0.546646
Batch:121.0
temp_loss: 0.643537
Batch:122.0
temp_loss: 0.557723
Batch:123.0
temp_loss: 0.57437
Batch:124.0
temp_loss: 0.488899
Batch:125.0
temp_loss: 0.693279
Batch:126.0
temp_loss: 0.518914
Batch:127.0
temp_loss: 0.556343
Batch:128.0
temp_loss: 0.556508
Batch:129.0
temp_loss: 0.531813
Batch:

temp_loss: 0.540497
Batch:346.0
temp_loss: 0.523974
Batch:347.0
temp_loss: 0.45905
Batch:348.0
temp_loss: 0.607236
Batch:349.0
temp_loss: 0.565132
Batch:350.0
temp_loss: 0.583113
Batch:351.0
temp_loss: 0.577544
validating...
validation loss:
0.54364697814
validation accuracy: 
0.727399981618
Batch:352.0
temp_loss: 0.581656
Batch:353.0
temp_loss: 0.637892
Batch:354.0
temp_loss: 0.618196
Batch:355.0
temp_loss: 0.571271
Batch:356.0
temp_loss: 0.497858
Batch:357.0
temp_loss: 0.714671
Batch:358.0
temp_loss: 0.493212
Batch:359.0
temp_loss: 0.424018
Batch:360.0
temp_loss: 0.494439
Batch:361.0
temp_loss: 0.486842
Batch:362.0
temp_loss: 0.704344
Batch:363.0
temp_loss: 0.631487
Batch:364.0
temp_loss: 0.628362
Batch:365.0
temp_loss: 0.564582
Batch:366.0
temp_loss: 0.585308
Batch:367.0
temp_loss: 0.559786
Batch:368.0
temp_loss: 0.525661
Batch:369.0
temp_loss: 0.59109
Batch:370.0
temp_loss: 0.544349
Batch:371.0
temp_loss: 0.474895
Batch:372.0
temp_loss: 0.541316
Batch:373.0
temp_loss: 0.705871
Batc

temp_loss: 0.564727
Batch:590.0
temp_loss: 0.40845
Batch:591.0
temp_loss: 0.505725
Batch:592.0
temp_loss: 0.52275
Batch:593.0
temp_loss: 0.533999
Batch:594.0
temp_loss: 0.366939
Batch:595.0
temp_loss: 0.667294
Batch:596.0
temp_loss: 0.512216
Batch:597.0
temp_loss: 0.64068
Batch:598.0
temp_loss: 0.363731
Batch:599.0
temp_loss: 0.586939
Batch:600.0
temp_loss: 0.564763
Batch:601.0
temp_loss: 0.44633
validating...
validation loss:
0.526203049421
validation accuracy: 
0.736999985576
Batch:602.0
temp_loss: 0.485367
Batch:603.0
temp_loss: 0.489309
Batch:604.0
temp_loss: 0.589519
Batch:605.0
temp_loss: 0.588444
Batch:606.0
temp_loss: 0.654573
Batch:607.0
temp_loss: 0.490953
Batch:608.0
temp_loss: 0.583552
Batch:609.0
temp_loss: 0.673958
Batch:610.0
temp_loss: 0.627158
Batch:611.0
temp_loss: 0.440058
Batch:612.0
temp_loss: 0.613835
Batch:613.0
temp_loss: 0.434207
Batch:614.0
temp_loss: 0.489296
Batch:615.0
temp_loss: 0.550502
Batch:616.0
temp_loss: 0.378749
Batch:617.0
temp_loss: 0.416558
Batch

temp_loss: 0.527457
Batch:211.0
temp_loss: 0.543646
Batch:212.0
temp_loss: 0.545483
Batch:213.0
temp_loss: 0.658907
Batch:214.0
temp_loss: 0.620889
Batch:215.0
temp_loss: 0.416444
Batch:216.0
temp_loss: 0.523353
Batch:217.0
temp_loss: 0.508455
Batch:218.0
temp_loss: 0.532795
Batch:219.0
temp_loss: 0.464229
Batch:220.0
temp_loss: 0.582315
Batch:221.0
temp_loss: 0.682112
Batch:222.0
temp_loss: 0.398827
Batch:223.0
temp_loss: 0.472192
Batch:224.0
temp_loss: 0.5428
Batch:225.0
temp_loss: 0.393802
Batch:226.0
temp_loss: 0.449747
Batch:227.0
temp_loss: 0.638001
Batch:228.0
temp_loss: 0.497257
Batch:229.0
temp_loss: 0.588959
Batch:230.0
temp_loss: 0.449432
Batch:231.0
temp_loss: 0.417605
Batch:232.0
temp_loss: 0.621673
Batch:233.0
temp_loss: 0.544571
Batch:234.0
temp_loss: 0.510316
Batch:235.0
temp_loss: 0.42349
Batch:236.0
temp_loss: 0.541772
Batch:237.0
temp_loss: 0.60166
Batch:238.0
temp_loss: 0.556124
Batch:239.0
temp_loss: 0.47739
Batch:240.0
temp_loss: 0.509307
Batch:241.0
temp_loss: 0.

temp_loss: 0.519735
Batch:456.0
temp_loss: 0.741979
Batch:457.0
temp_loss: 0.434042
Batch:458.0
temp_loss: 0.49701
Batch:459.0
temp_loss: 0.440221
Batch:460.0
temp_loss: 0.469165
Batch:461.0
temp_loss: 0.458903
Batch:462.0
temp_loss: 0.487078
Batch:463.0
temp_loss: 0.514317
Batch:464.0
temp_loss: 0.521819
Batch:465.0
temp_loss: 0.553361
Batch:466.0
temp_loss: 0.518508
Batch:467.0
temp_loss: 0.551847
Batch:468.0
temp_loss: 0.448089
Batch:469.0
temp_loss: 0.578237
Batch:470.0
temp_loss: 0.464826
Batch:471.0
temp_loss: 0.553343
Batch:472.0
temp_loss: 0.429198
Batch:473.0
temp_loss: 0.574013
Batch:474.0
temp_loss: 0.592911
Batch:475.0
temp_loss: 0.484374
Batch:476.0
temp_loss: 0.336723
Batch:477.0
temp_loss: 0.441752
Batch:478.0
temp_loss: 0.402659
Batch:479.0
temp_loss: 0.414246
Batch:480.0
temp_loss: 0.464162
Batch:481.0
temp_loss: 0.388207
Batch:482.0
temp_loss: 0.400858
Batch:483.0
temp_loss: 0.641007
Batch:484.0
temp_loss: 0.474011
Batch:485.0
temp_loss: 0.689178
Batch:486.0
temp_loss

KeyboardInterrupt: 